In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import gc
import time
import warnings

os.chdir(r'C:\Users\INHA\iCloudDrive\FinanceLab\Paper\replicate\Gu_et_al_2020\data')

warnings.filterwarnings('ignore')

%matplotlib inline



### Data Preparation

Firm Characteristics Data

The newest version of firm characteristics data are downloaded from 

In [4]:
%%time
#start date and end date of the sample
start_date, end_date = 19570531, 20161231

#load firm characteristics data
data_ch = pd.read_csv('GKX_20201231.csv')
data_ch = data_ch[(data_ch['DATE']>=start_date)&(data_ch['DATE']<=end_date)].reset_index(drop=True)
data_ch['DATE'] = pd.to_datetime(data_ch['DATE'], format='%Y%m%d')+pd.offsets.MonthEnd(0)
characteristics = list(set(data_ch.columns).difference({'permno','DATE','SHROUT','mve0','sic2','RET','prc'}))

data_ch.head()

CPU times: total: 1min 20s
Wall time: 6min 27s


,permno,DATE,mvel1,RET,prc,SHROUT,beta,betasq,chmom,dolvol,...,baspread,ill,maxret,retvol,std_dolvol,std_turn,zerotrade,sic2,bm,bm_ia
0,10006,1957-05-31,87191.00,0.004049,61.000,1412,1.105519,1.222172,0.206966,9.224909,...,0.008677,1.582667e-07,0.020576,0.008402,0.595904,0.403419,1.407496e-07,37.0,NaN,NaN
1,10014,1957-05-31,4089.25,0.272727,3.500,1487,0.353767,0.125151,-0.012857,6.335719,...,0.038108,2.006570e-05,0.050000,0.040580,0.800049,0.594928,1.548958e-07,NaN,NaN,NaN
2,10022,1957-05-31,10219.50,0.037037,7.000,1514,0.955740,0.913438,0.127109,6.631508,...,0.015120,3.056159e-06,0.019608,0.012231,0.997025,0.621414,1.000000e+00,NaN,NaN,NaN
3,10030,1957-05-31,56729.00,0.029255,48.375,1207,0.875973,0.767328,0.187076,9.827079,...,0.010752,3.829632e-08,0.016216,0.008002,0.496714,1.062983,4.088753e-08,NaN,NaN,NaN
4,10057,1957-05-31,37000.00,-0.027027,71.000,500,1.160347,1.346405,0.045401,7.415777,...,0.009028,7.115245e-07,0.017123,0.011561,0.633949,0.302686,3.000000e+00,NaN,NaN,NaN


### Pick out Top 1000 and Bottom 1000 Firms

Next, let's pick out the top 1000 and bottom 1000 firms with respect to market capitalization to see the difference of predictability between big firms and small firms

In [13]:
#Pick top 1000 firms by market value each year
data_ch_top = data_ch.sort_values('mvel1', ascending=False).groupby('DATE').head(1000).reset_index(drop=True)
#Pcik bottom 1000 firms by market value each year
data_ch_bot = data_ch.sort_values('mvel1', ascending=False).groupby('DATE').head(1000).reset_index(drop=True)

### Missing Characteristics

According to thwe paper, the missing data are replaced by the cross-sectional median

In [6]:
#missing data before filling
data_ch.isnull().sum()

permno            0
DATE              0
mvel1          3038
RET               0
prc           19070
              ...  
std_turn     305622
zerotrade    309732
sic2         257771
bm           989890
bm_ia        989890
Length: 101, dtype: int64

In [7]:
%%time
#fill nan with cross-sectional median
for ch in characteristics:
    data_ch[ch] = data_ch.groupby('DATE')[ch].transform(lambda x: x.fillna(x.median()))

CPU times: total: 2min 6s
Wall time: 3min 8s


In [8]:
#missing data after filling
data_ch.isnull().sum()

permno            0
DATE              0
mvel1             0
RET               0
prc           19070
              ...  
std_turn          0
zerotrade         0
sic2         257771
bm            67854
bm_ia         67854
Length: 101, dtype: int64

Since there are some characeristics that are all missing at some time point, we still encounter missing data after the filling process. Then, let's try to fill the remaining nan with time-series median. Unfortunately, after filling nan with time-series median, na still exists. 
Since there is no further instruction of how to deal with remaining nan in the data, after consulting some replication code online, I fill the remaining nan with 0.

In [9]:
for ch in characteristics:
    data_ch[ch] = data_ch[ch].fillna(0)

data_ch.columns[data_ch.isnull().sum()!=0]

Index(['prc', 'mve0', 'sic2'], dtype='object')

Now, we do not have missing characteristics in dataset.

Then, do the same process to top and bottom 1000 firms data.

In [20]:
def fill_nan(data_ch, characteristics):
    for ch in characteristics:
        data_ch[ch] = data_ch.groupby('DATE')[ch].transform(lambda x: x.fillna(x.median()))
    for ch in characteristics:
        data_ch[ch] = data_ch[ch].fillna(0)
    return data_ch

In [21]:
data_ch_top = fill_nan(data_ch_top, characteristics)
data_ch_bot = fill_nan(data_ch_bot, characteristics)

### Transform SIC Code into Dummies

In [24]:
#get dummies for SIC code
def get_sic_dummies(data_ch):
    sic_dummies = pd.get_dummies(data_ch['sic2'].fillna(999).astype(int), prefix='sic').drop('sic_999', axis=1)
    data_ch_d = pd.concat([data_ch, sic_dummies], axis=1)
    data_ch_d.drop(['prc', 'SHROUT', 'mve0', 'sic2'], inplace=True, axis=1)
    return data_ch_d

In [25]:
data_ch_d = get_sic_dummies(data_ch)
data_ch_top_d = get_sic_dummies(data_ch_top)
data_ch_bot_d = get_sic_dummies(data_ch_bot)

### Macroeconomic Predictors Data

The eight macroeconomic predictors follows the definitions by Welch and Goyal (2008, RFS). The data are available on Prof Goyal's website.

In [33]:
#loead macroeconomic predictors data
data_ma = pd.read_csv('PredictorData2022.xlsx - Monthly.csv')
data_ma = data_ma[(data_ma['yyyymm']>=start_date//100) & (data_ma['yyyymm']<=end_date//100)].reset_index(drop=True)

# construct predictor
ma_predictors = ['dp_sp', 'ep_sp', 'bm_sp', 'ntis', 'tbl', 'tms', 'dfy', 'svar']
data_ma['Index'] = data_ma['Index'].str.replace(',', '').astype('float64')
data_ma['dp_sp'] = data_ma['D12']/data_ma['Index']
data_ma['ep_sp'] = data_ma['E12']/data_ma['Index']

data_ma.rename({'b/m': 'bm_sp'}, axis=1, inplace=True)

data_ma['tms'] = data_ma['lty']-data_ma['tbl']
data_ma['dfy'] = data_ma['BAA']-data_ma['AAA']

data_ma = data_ma[['yyyymm']+ma_predictors]
data_ma['yyyymm'] = pd.to_datetime(data_ma['yyyymm'], format='%Y%m')+pd.offsets.MonthEnd(0)
data_ma.head()

,yyyymm,dp_sp,ep_sp,bm_sp,ntis,tbl,tms,dfy,svar
0,1957-05-31,0.036475,0.071965,0.564039,0.028849,0.0306,0.0042,0.0078,0.000482
1,1957-06-30,0.036521,0.072198,0.565877,0.030528,0.0329,0.0032,0.0072,0.000579
2,1957-07-31,0.036318,0.071732,0.560057,0.032346,0.0316,0.0049,0.0074,0.000554
3,1957-08-31,0.038700,0.076367,0.588005,0.035854,0.0337,0.0030,0.0072,0.002147
4,1957-09-30,0.041490,0.081801,0.624151,0.034363,0.0353,0.0011,0.0081,0.001517


### Construct the Dataset including all the Features

Besides adding the interaction terms, this function also transform the data into (-1,1).

In [52]:
from sklearn.preprocessing import MinMaxScaler

def interactions(data_ch, data_ma, chracteristics, ma_predictors, minmax=True):
    # construct interactions between firm characteristics and macroeconomic predictors
    data = data_ch.copy()
    data_ma_long = pd.merge(data[['DATE']], data_ma, left_on='DATE', right_on='yyyymm', how='left')
    data = data.reset_index(drop=True)
    data_ma_long = data_ma_long.reset_index(drop=True)
    for fc in characteristics:
        for mp in ma_predictors:
            data[fc+'*'+mp] = data[fc]*data_ma_long[mp]
            
    features = list(set(data.columns).difference({'permo', 'DATE', 'RET'})) # a list storing all 920 features used
    
    if minmax:
       X = MinMaxScaler((-1,1)).fit_transform(data[features])
       X = pd.DataFrame(X, columns=features)
    else:
        X = data[features]
    y = data['RET']
    print(f"The shape of the data is {data.shape}")
    return X, y

### Split the Sample into Training Set, Validation Set and Testing Set

According to the paper, the authors use first 18 years (1957-1974) for training, last 30 years (19878-2016), for out-of-sample testing, and 12 years middle(1975-1986) for tuning hyperparameters.

In [53]:
start_date_valid = np.datetime64('1975-01-31')
start_date_test = np.datetime64('1987-01-31')

In [54]:
def train_valid_test(data):
    
    #training set_start_date_valid = np.datetime64('1957-05-31')
    X_train, y_train = interactions(data[data['DATE']<start_date_valid], data_ma[data_ma['yyyymm']<start_date_valid], characteristics, ma_predictors)
    
    #validation set
    X_valid, y_valid = interactions(data[(data['DATE']<start_date_test)&(data['DATE']>=start_date_valid)], data_ma[(data_ma['yyyymm']>=start_date_valid)], characteristics, ma_predictors)
    
    #test set
    X_test, y_test = interactions(data[data['DATE']>=start_date_test], data_ma[data_ma['yyyymm']>=start_date_test], characteristics,ma_predicors)
    
    return X_train, X_valid, X_test, y_train, y_valid, y_test

In [55]:
%%time

X_train, X_valid, X_test, y_train, y_valid, y_test = train_valid_test(data_ch_top_d)

MemoryError: Unable to allocate 1.45 GiB for an array with shape (915, 212000) and data type float64